In [1]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from typing import Literal

from dotenv import load_dotenv
load_dotenv(dotenv_path='D:/GenAI-Practice/AgenticAI-Projects/SmartAssistant/.env')

# Initialize model (replace with your working one, e.g., gemini-2.5-pro)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                             temperature=0,
                             api_key=os.getenv('GOOGLE_API_KEY'))

d:\GenAI-Practice\env_RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classifier Node

In [2]:
from pydantic import BaseModel, Field
from typing import Optional, Literal, List, Any

class ChatbotState(BaseModel):
    """Represents the state of the chatbot flow."""
    
    query: str = Field(..., description="User's current query")
    query_type: Optional[Literal["llm", "rag", "db"]] = Field(
        default=None, description="Type of query determined by classifier")
    context: Optional[List[str]] = Field(
        default=None, description="Retrieved context or documents for RAG")
    answer: Optional[str] = Field(
        default=None, description="Final generated answer for user")
    reasoning_trace: Optional[str] = Field(
        default=None, description="Explanation of reasoning or tool selection")
    memory: Optional[Any] = Field(
        default=None, description="Conversation memory (optional future use)")


In [3]:
# Structured output schema for LLM classification
class QueryClassification(BaseModel):
    query_type: Literal["llm", "rag", "db"] = Field(
        ..., description="Type of query (llm, rag, or db)")
    reasoning: str = Field(..., description="Explanation of why this classification was made")

# Define parser
parser = PydanticOutputParser(pydantic_object=QueryClassification)

# Define prompt
prompt = PromptTemplate(
    template=(
        "You are a query classification assistant. "
        "Given a user question, determine if it should be handled by:\n"
        "- 'llm': for general world knowledge or reasoning questions.\n"
        "- 'rag': for document-based or semantic search questions.\n"
        "- 'db': for database or structured data retrieval queries.\n\n"
        "Return a structured JSON matching this schema:\n{format_instructions}\n\n"
        "Question: {query}"
    ),
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

def classify_query_node(state: ChatbotState) -> ChatbotState:
    """Classify query into llm, rag, or db using Gemini and structured output."""
    try:
        input_prompt = prompt.format_prompt(query=state.query)
        response = llm.invoke(input_prompt.to_string())
        parsed = parser.parse(response.content)

        state.query_type = parsed.query_type
        state.reasoning_trace = parsed.reasoning
        return state

    except Exception as e:
        state.query_type = "llm"  # fallback
        state.reasoning_trace = f"Classification failed: {e}"
        return state

Database Node

In [4]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit
from langchain_google_genai import ChatGoogleGenerativeAI

def db_query_node(state: ChatbotState) -> ChatbotState:
    """
    Handles structured data queries via LangChain SQL Agent.
    Executes natural language questions on database and updates the state.
    """
    try:
        # Load database
        db = SQLDatabase.from_uri("sqlite:///D:/GenAI-Practice/AgenticAI-Projects/SmartAssistant/app/data/uploads/bank_domain.db")

        # Create toolkit for SQL agent
        toolkit = SQLDatabaseToolkit(db=db, llm=ChatGoogleGenerativeAI(model="gemini-2.5-pro"))

        # Create the SQL agent
        agent_executor = create_sql_agent(
            llm=toolkit.llm,
            toolkit=toolkit,
            verbose=True,
            agent_type="openai-tools",  # works well with Gemini too
        )

        # Run query
        query = state.query
        response = agent_executor.invoke({"input": query})

        # Update chatbot state
        state.answer = response["output"]
        state.reasoning_trace = (
            f"Used SQL Agent on 'sample.db' to execute query derived from user question."
        )
        return state

    except Exception as e:
        state.answer = f"Error during DB query: {e}"
        state.reasoning_trace = "DB Node failed to process query."
        return state


RAG Node

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
# from core.embedding_utils import get_huggingface_embedding

import os
from langchain_community.embeddings import HuggingFaceEmbeddings

def get_huggingface_embedding(model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
    """
    Initialize and return a Hugging Face embedding model.
    Args:
        model_name: Name of the Hugging Face model to use.
    Returns:
        HuggingFaceEmbeddings instance or None if error occurs.
    """
    try:
        print(f"🔹 Initializing Hugging Face Embedding model: {model_name}")
        embeddings = HuggingFaceEmbeddings(model_name=model_name)
        return embeddings
    except Exception as e:
        print(f"⚠️ Error initializing Hugging Face Embedding model: {e}")
        return None

# from chatbot.state import ChatbotState

def rag_node(state: ChatbotState) -> ChatbotState:
    """
    Handles document-based queries using FAISS + LLM.
    Retrieves relevant chunks and generates an answer.
    """
    try:
        # Step 1: Load embeddings and FAISS vector store
        embeddings = get_huggingface_embedding()
        vectorstore = FAISS.load_local("D:/GenAI-Practice/AgenticAI-Projects/SmartAssistant/app/data/faiss_index", embeddings, allow_dangerous_deserialization=True)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

        # Step 2: Create LLM (Gemini or fallback)
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)

        # Step 3: Create Retrieval-QA chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )

        # Step 4: Run the query
        result = qa_chain.invoke({"query": state.query})

        # Step 5: Update state
        state.answer = result["result"]
        state.context = [doc.page_content for doc in result["source_documents"]]
        state.reasoning_trace = (
            "Used FAISS semantic search to find relevant chunks, then used Gemini for synthesis."
        )
        return state

    except Exception as e:
        state.answer = f"RAG failed: {e}"
        state.reasoning_trace = "RAG node failed to process query."
        return state

LLM Node

In [6]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
# from chatbot.state import ChatbotState
# LLM = ChatGoogleGenerativeAI(model="gemini-2.5-pro", api_key=os.getenv('GOOGLE_API_KEY'))

def llm_node(state: ChatbotState) -> ChatbotState:
    """
    Handles open-domain or reasoning queries directly using Gemini.
    """
    try:
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.7,  api_key=os.getenv('GOOGLE_API_KEY'))
        response = llm.invoke(state.query)

        state.answer = response.content
        state.reasoning_trace = "Used Gemini directly for general reasoning or world knowledge question."
        return state

    except Exception as e:
        state.answer = f"LLM node failed: {e}"
        state.reasoning_trace = "LLM node could not generate an answer."
        return state

Graph

In [11]:
from langgraph.graph import StateGraph, END

def router(state: ChatbotState) -> str:
    """
    Router node — decides which branch to take based on query_type.
    Returns one of: 'rag', 'db', 'llm'
    """
    if state.query_type == "rag":
        return "rag"
    elif state.query_type == "db":
        return "db"
    else:
        return "llm"


def build_chatbot_graph() -> StateGraph:
    """
    Build the chatbot LangGraph pipeline with classification + routing.
    Compatible with latest LangGraph API.
    """
    graph = StateGraph(ChatbotState)

    # --- Define nodes ---
    graph.add_node("classify", classify_query_node)
    graph.add_node("rag", rag_node)
    graph.add_node("db", db_query_node)
    graph.add_node("llm", llm_node)

    # --- Edges ---
    # First step always goes to classification
    graph.set_entry_point("classify")

    # Conditional routing
    graph.add_conditional_edges(
        "classify",
        router,  # function that returns key: rag / db / llm
        {
            "rag": "rag",
            "db": "db",
            "llm": "llm",
        },
    )

    # --- End edges ---
    graph.add_edge("rag", END)
    graph.add_edge("db", END)
    graph.add_edge("llm", END)

    graph.set_entry_point("classify")

    app = graph.compile()

    return app

# Initialize chatbot graph
chatbot_graph = build_chatbot_graph()

In [12]:
# Example invocation helper
def run_chatbot(query: str) -> ChatbotState:
    """
    Run the query through the LangGraph pipeline.
    Returns the final ChatbotState with answer, context, reasoning_trace.
    """
    initial_state = ChatbotState(query=query)
    result_state = chatbot_graph.invoke(initial_state)
    return result_state

In [13]:
run_chatbot(query = 'How many bank branches are there?')



> Entering new SQL Agent Executor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 56.81594666s. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 56
}
].



Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded:  Finally I should write a query to answer the question about the number of bank branches.

accounts, branch_locations, credit_cards, credit_exposures, customers, employees, insurance_policies, investments, loans, transactions

{'query': 'How many bank branches are there?',
 'query_type': 'db',
 'answer': 'Error during DB query: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2\nPlease retry in 54.626325533s. [violations {\n}\n, links {\n  description: "Learn more about Gemini API quotas"\n  url: "https://ai.google.dev/gemini-api/docs/rate-limits"\n}\n, retry_delay {\n  seconds: 54\n}\n]',
 'reasoning_trace': 'DB Node failed to process query.'}

In [14]:
run_chatbot(query = 'What is capital of US?')

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 22.601202968s. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 22
}
].


{'query': 'What is capital of US?',
 'query_type': 'llm',
 'answer': 'LLM node failed: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2\nPlease retry in 20.432528054s. [violations {\n}\n, links {\n  description: "Learn more about Gemini API quotas"\n  url: "https://ai.google.dev/gemini-api/docs/rate-limits"\n}\n, retry_delay {\n  seconds: 20\n}\n]',
 'reasoning_trace': 'LLM node could not generate an answer.'}

In [18]:
response = run_chatbot(query = 'Can you tell me what is encoder and decoder from the document?')

🔹 Initializing Hugging Face Embedding model: sentence-transformers/all-MiniLM-L6-v2


In [22]:
print(response['answer'])

Based on the document provided:

**Encoder:**
The encoder's role is to map an input sequence of symbol representations (x1, ..., xn) into a sequence of continuous representations, z = (z1, ..., zn). It contains self-attention layers where each position in the encoder can attend to all positions from the previous layer.

**Decoder:**
The decoder's role is to take the continuous representation **z** from the encoder and generate an output sequence of symbols (y1, ..., ym) one element at a time. It is "auto-regressive," meaning it uses the symbols it has already generated as additional input to generate the next one.

The decoder has three main sub-layers:
1.  A modified self-attention layer that prevents positions from attending to subsequent positions (masking).
2.  An "encoder-decoder attention" layer where it performs multi-head attention over the output of the encoder. This allows every position in the decoder to attend to all positions in the input sequence.
3.  A third sub-layer (i